<a href="https://colab.research.google.com/github/Balrog16/Test/blob/master/MIMII_Fan_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Background**

Fan audio data from MIMII database is played after combining a few files randomly to form two audio records - normal and abnormal, each lasting for 520 sec. This data is played on JBL Go speaker with SITRANS-MS placed in the middle of the speaker. This arrangement is to vibrate the sensor and obtain the stream via BLE using MATLAB scripts. There are two sets of JSON files each for Normal audio and abnormal audio data.


In [ ]:
# all imports
import json
import numpy as np
import tensorflow as tf
from numpy import savetxt

In [ ]:
class SMSData:
    def __init__(self, fileName, label):
        self.fileName = fileName
        f = open(self.fileName)
        data = json.load(f)
        self.accelX = np.array(data['values']['5FF8'])
        self.accelY = np.array(data['values']['5FF9'])
        self.accelZ = np.array(data['values']['5FFA'])
        # Collapse the last two columns to one by 2-norm of the components
        self.accelX = np.sqrt(np.square(self.accelX[:,:,0]) + np.square(self.accelX[:, :, 1]))
        self.accelY = np.sqrt(np.square(self.accelY[:,:,0]) + np.square(self.accelY[:, :, 1]))
        self.accelZ = np.sqrt(np.square(self.accelZ[:,:,0]) + np.square(self.accelZ[:, :, 1]))
        # Close the file
        f.close()
        self.minCols = (min(self.accelX.shape[1], self.accelY.shape[1], self.accelZ.shape[1]))
        self.accelX  = np.transpose(self.accelX[:, 0: self.minCols])
        self.accelY  = np.transpose(self.accelY[:, 0: self.minCols])
        self.accelZ  = np.transpose(self.accelZ[:, 0: self.minCols])
        self.yData   = np.ones([self.accelY.shape[0], 1])*label
        self.featVecA = self.accelX + self.accelY + self.accelZ 
        self.featVecB  = np.dstack((self.accelX, self.accelY, self.accelZ))


In [ ]:
# create empty objects
SMSDataObjNormal_1   = []
SMSDataObjNormal_2   = []
SMSDataObjabNormal_1 = []
SMSDataObjabNormal_2 = []

# define a class object to hold the data
SMSDataObjNormal_1 = SMSData("/content/normal_1_6dB_Fan.json", 0)
SMSDataObjNormal_2 = SMSData("/content/normal_2_6dB_Fan.json", 0)
SMSDataObjabNormal_1 = SMSData("/content/sample_data/abnormal_1_6dB_Fan.json", 1)
SMSDataObjabNormal_2 = SMSData("/content/sample_data/abnormal_2_6dB_Fan.json", 1)


In [ ]:
# for debug purpose
print('No of columns used', SMSDataObjabNormal_1.minCols)
print('Size of each axis', np.shape(SMSDataObjabNormal_1.accelX))
print('Feature vector B looks like', np.shape(SMSDataObjabNormal_1.featVecB))
minCols = min(SMSDataObjNormal_1.minCols, SMSDataObjNormal_2.minCols, SMSDataObjabNormal_1.minCols, SMSDataObjabNormal_1.minCols)
print('Minimum cols across the data is', minCols)

No of columns used 513
Size of each axis (513, 256)
Feature vector B looks like (513, 256, 3)
Minimum cols across the data is 512


In [ ]:
trainIdx = 451
testIdx = 451
endIdx = minCols
# generate a dataset for training and testing
x_train = np.concatenate((SMSDataObjNormal_1.featVecB[1:trainIdx, :, :], 
                          SMSDataObjabNormal_1.featVecB[1:trainIdx, :, :], 
                          SMSDataObjNormal_2.featVecB[1:trainIdx, :, :], 
                          SMSDataObjabNormal_2.featVecB[1:trainIdx, :, :]))

y_train = np.concatenate((SMSDataObjNormal_1.yData[1:trainIdx, :], 
                          SMSDataObjabNormal_1.yData[1:trainIdx, :], 
                          SMSDataObjNormal_2.yData[1:trainIdx, :], 
                          SMSDataObjabNormal_2.yData[1:trainIdx, :]))

x_test = np.concatenate((SMSDataObjNormal_1.featVecB[testIdx:endIdx, :, :], 
                          SMSDataObjNormal_2.featVecB[testIdx:endIdx, :, :], 
                          SMSDataObjabNormal_1.featVecB[testIdx:endIdx, :, :], 
                          SMSDataObjabNormal_2.featVecB[testIdx:endIdx, :, :]))

y_test = np.concatenate((SMSDataObjNormal_1.yData[testIdx:endIdx, :], 
                          SMSDataObjNormal_2.yData[testIdx:endIdx, :], 
                          SMSDataObjabNormal_1.yData[testIdx:endIdx, :], 
                          SMSDataObjabNormal_2.yData[testIdx:endIdx, :]))

In [ ]:
# check vector shape
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)


(1800, 256, 3)
(1800, 1)
(244, 256, 3)
(244, 1)


In [69]:
# time to try NN

# model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(256, 3)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2)
])

# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# settings
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [70]:
# train the dog
model.fit(x_train, y_train, epochs=25)

Epoch 1/25
57/57 [==============================] - 0s 1ms/step - loss: 5.8309 - accuracy: 0.5467
Epoch 2/25
57/57 [==============================] - 0s 1ms/step - loss: 2.3241 - accuracy: 0.6883
Epoch 3/25
57/57 [==============================] - 0s 1ms/step - loss: 0.9347 - accuracy: 0.7900
Epoch 4/25
57/57 [==============================] - 0s 1ms/step - loss: 0.3983 - accuracy: 0.8800
Epoch 5/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1924 - accuracy: 0.9289
Epoch 6/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1301 - accuracy: 0.9511
Epoch 7/25
57/57 [==============================] - 0s 1ms/step - loss: 0.1020 - accuracy: 0.9656
Epoch 8/25
57/57 [==============================] - 0s 1ms/step - loss: 0.0783 - accuracy: 0.9711
Epoch 9/25
57/57 [==============================] - 0s 1ms/step - loss: 0.0681 - accuracy: 0.9778
Epoch 10/25
57/57 [==============================] - 0s 1ms/step - loss: 0.0598 - accuracy: 0.9767
Epoch 11/25
57/57 [

In [71]:
# ask him to perform tricks
model.evaluate(x_test[200:240],  y_test[200:240], verbose=2)

2/2 - 0s - loss: 0.0663 - accuracy: 0.9750


[0.06627007573843002, 0.9750000238418579]

In [72]:
# what's the probability?
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[1:3])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9.9999142e-01, 8.5378078e-06],
       [9.9997568e-01, 2.4284695e-05]], dtype=float32)>

In [78]:
predictNow = model(x_test[120:121]).numpy()
predictNow

array([[-12.55127 , -13.230565]], dtype=float32)